# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 22 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 23). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la energía generada 24 horas después. 


## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.

In [1]:
import pandas as pd
import numpy as np
# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')
# FIlter the data to only include the columns that end in 13
import time
a = time.time()
data = data.filter(regex='13$|energy')
#print(data)
print(time.time()-a)



0.006834268569946289


In [2]:
from sklearn.model_selection import train_test_split

y = data['energy']
x = data.drop(columns=['energy'])


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7543)


In [3]:
# Preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier  
from sklearn.pipeline import Pipeline

scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer()]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), x.columns)
    ])


In [4]:
# Optimizing hyperparameters
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor  # Import DecisionTreeRegressor instead of DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
import numpy as np


param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Adjust criterion for regression
    'max_depth': [None, 12, 13, 14, 15, 16, 17],
    'min_samples_split': range(2, 15, 2),
    'min_samples_leaf': range(5, 15, 2),
}

cv = KFold(n_splits=5)
model = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error', n_iter=50)  # Adjust scoring metric

model2 = GridSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


rmse_tree = metrics.mean_squared_error(y_test, y_pred)
sqrt_rmse_tree = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse_tree}")
print(f"SQRT RMSE: {sqrt_rmse_tree}")
print(f"Best params: {model.best_params_}")
# Best params: {'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 3}


Fitting 5 folds for each of 50 candidates, totalling 250 fits
RMSE: 167902.30422607897
SQRT RMSE: 409.7588366662505
Best params: {'min_samples_split': 4, 'min_samples_leaf': 11, 'max_depth': 13, 'criterion': 'absolute_error'}


In [12]:
# Create a Dunny regressor to obtain a baseline

from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', dummy)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

rmse_dummy = metrics.mean_squared_error(y_test, y_pred)
sqrt_rmse_dummy = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse_dummy}")
print(f"SQRT RMSE: {sqrt_rmse_dummy}")



RMSE: 451489.08466268575
SQRT RMSE: 671.9293747580067


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV


param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1],  # -1 indica utilizar todos los núcleos
    'positive': [True, False]
}


# Crear el modelo de regresión lineal
model = LinearRegression()

grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           scoring="neg_mean_absolute_percentage_error"
                           )

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)



C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.41046e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.21305e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.12081e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.36438e-17): result may not be accura

Mejor modelo: LinearRegression()
Mejorres parámetros: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}
RMSE: 559.7095305382695


C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=1.14809e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=1.02082e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:145: LinAlgWarning: Ill-conditioned matrix (rcond=1.35645e-17): result may not be accurate.
  s[P] = solve(AtA[np.ix_(P, P)], Atb[P], assume_a='sym',
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=1.08655e-17): result may not be accurate

Ahora que ya hemos probado con el Lineal Model normal, vamos a probar con la variante de Lasso a ver si conseguimos mejorar el entrenamiento:

In [20]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Definir los parámetros que deseas probar
parameters = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Valores de regularización
    'fit_intercept': [True, False],
    'copy_X': [True, False], # pendientes de borrar
    'positive': [True, False], 
    'precompute': [True, False], 
    'random_state': [7543], 
    'selection': ['cyclic', 'random'], 
    'tol': [1**(-10), 1**(-5), 1**(-4), 1**(-3), 1**(-2)], 
    'warm_start': [True, False]
}

# Crear el modelo de regresión lineal Lasso
model = Lasso()

grid_search = GridSearchCV(model, 
                           parameters, 
                           scoring='neg_mean_squared_error', 
                           cv=5)

grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)

C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6400 fits failed out of a total of 44800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\Admi

Mejor modelo: Lasso(alpha=0.1, max_iter=12, random_state=7543, selection='random', tol=1.0,
      warm_start=True)
Mejorres parámetros: {'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': 12, 'positive': False, 'precompute': False, 'random_state': 7543, 'selection': 'random', 'tol': 1.0, 'warm_start': True}
RMSE: 567.7651275011757


C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [             nan              nan              nan ... -374582.31832023
 -374582.31832023 -374582.31832023]
  warnings.warn(


In [6]:
# Vamos a probar con KNN Regresor
from sklearn import neighbors
from sklearn import metrics
regr = neighbors.KNeighborsRegressor()

# We train it
regr.fit(X_train, y_train)

# We obtain predictions on the test set
y_test_pred = regr.predict(X_test)

# We compute accuracy
rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print(f"RMSE of the model: {rmse_knn}")

RMSE of the model: 621.2880027885399
